In [ ]:
%%HTML
<style type="text/css">
table.dataframe td, table.dataframe th {
    border: 1px  white solid !important;
  color: white !important;
}

In [ ]:
%matplotlib inline

from bs4 import BeautifulSoup
from iexfinance.stocks import Stock

#import iexfinance
import matplotlib
import matplotlib.pyplot as plt
import re
import requests
import numpy as np
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.webdriver import WebDriver
import warnings


warnings.filterwarnings('ignore')

In [ ]:
def get_table(soup):
    for t in soup.select('table'):
        header = t.select('thead tr th')
        if len(header) > 2:
            if (header[0].get_text().strip() == 'Symbol'
                and header[2].get_text().strip().startswith('% Holding')):
                return t
    raise Exception('could not find symbol list table')


def get_etf_holdings(etf_symbol):
    '''
    etf_symbol: str
    
    return: pd.DataFrame

    '''
    
    

    url = 'https://www.barchart.com/stocks/quotes/{}/constituents?page=all'.format(
        etf_symbol)
    # Loads the ETF constituents page and reads the holdings table
    browser = WebDriver() # webdriver.PhantomJS()
    browser.get(url)
    html = browser.page_source
    soup = BeautifulSoup(html, 'html')
    table = get_table(soup)
    # Reads the holdings table line by line and appends each asset to a
    # dictionary along with the holdings percentage
    asset_dict = {}
    for row in table.select('tr')[1:-1]:
        try:
            cells = row.select('td')
        
            symbol = cells[0].get_text().strip()
            name = cells[1].text.strip()
            celltext = cells[2].get_text().strip()
            percent = celltext.rstrip('%')
            shares = cells[3].text.strip().replace(',', '')
            if symbol != "" and percent != 0.0:
                asset_dict[symbol] = {
                    'name': name,
                    'percent': percent,
                    'shares': shares,
                }
            output = pd.DataFrame(asset_dict).T.reset_index().rename(columns={'index':'symbol'})
            
        except BaseException as ex:
            print(ex)
    browser.quit()

    return output

In [ ]:

spyd = get_etf_holdings('SPYD')

In [ ]:
spyd.to_dict('records')